In [2]:
import feedparser
from datetime import datetime

RSS_URL = "https://www.ccomptes.fr/fr/rss/general"
feed = feedparser.parse(RSS_URL)

rapports = []
for entry in feed.entries:
    # Filtre 1 : uniquement les publications (pas /actualites/, /recrutement/...)
    if "/publications/" not in entry.link:
        continue

    # Filtre 2 : post-2020
    year = entry.published_parsed.tm_year
    if year < 2020:
        continue

    rapports.append({
        "title": entry.title,
        "url": entry.link,
        "year": year
    })

print(f"{len(rapports)} rapports trouvés")
for r in rapports:
    print(r)

5 rapports trouvés
{'title': 'Commune de Frasseto (Corse-du-Sud)', 'url': 'https://www.ccomptes.fr/fr/publications/commune-de-frasseto-corse-du-sud', 'year': 2026}
{'title': "Commune d'Ajaccio (Corse-du-Sud)", 'url': 'https://www.ccomptes.fr/fr/publications/commune-dajaccio-corse-du-sud-8', 'year': 2026}
{'title': 'Département de Seine-Saint-Denis (avis budgétaire)', 'url': 'https://www.ccomptes.fr/fr/publications/departement-de-seine-saint-denis-avis-budgetaire', 'year': 2026}
{'title': 'La situation des finances publiques début 2026', 'url': 'https://www.ccomptes.fr/fr/publications/la-situation-des-finances-publiques-debut-2026', 'year': 2026}
{'title': 'La rémunération à la performance des agents de l’État', 'url': 'https://www.ccomptes.fr/fr/publications/la-remuneration-la-performance-des-agents-de-letat', 'year': 2026}


In [3]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://www.igf.finances.gouv.fr"
PAGE_URL = f"{BASE_URL}/liste-de-tous-les-rapports-de-mi.html"

response = requests.get(PAGE_URL)
soup = BeautifulSoup(response.text, "html.parser")

# Trouver tous les liens PDF
liens_pdf = soup.find_all("a", href=lambda h: h and ".pdf" in h.lower())

print(f"{len(liens_pdf)} liens PDF trouvés")
for lien in liens_pdf[:10]:
    print(lien.get("href"), "|", lien.get("title", ""))

25 liens PDF trouvés
/files/live/sites/igf/files/contributed/Rapports%20de%20mission/2026/2025-E-023%20Synth%c3%a8se.pdf | Face à la gravité de la situation financière des hôpitaux publics, renforcer l'efficience par une intégration territoriale
/files/live/sites/igf/files/contributed/Rapports%20de%20mission/2026/2025-E-023%20Rapport.pdf | Face à la gravité de la situation financière des hôpitaux publics, renforcer l'efficience par une intégration territoriale
/files/live/sites/igf/files/contributed/Rapports%20de%20mission/2026/2025-E-023%20Annexe%201.pdf | 2025-E-023 Annexe 1.pdf
/files/live/sites/igf/files/contributed/Rapports%20de%20mission/2026/2025-E-023%20Annexe%202.pdf | Face à la gravité de la situation financière des hôpitaux publics, renforcer l'efficience par une intégration territoriale
/files/live/sites/igf/files/contributed/Rapports%20de%20mission/2026/2025-E-023%20Annexe%203.pdf | Face à la gravité de la situation financière des hôpitaux publics, renforcer l'efficience p

In [ ]:
from urllib.parse import unquote

rapports = []

for lien in liens_pdf:
    href = lien.get("href", "")
    title = lien.get("title", "")

    # Filtre 1 : garder uniquement les rapports principaux
    # (pas Synthèse, pas Annexe)
    nom_fichier = unquote(href.split("/")[-1]).lower()
    if "annexe" in nom_fichier or "synth" in nom_fichier:
        continue

    # Filtre 2 : extraire l'année depuis l'URL
    # /Rapports de mission/2026/... → 2026
    parties = href.split("/")
    year = None
    for partie in parties:
        if partie.isdigit() and len(partie) == 4:
            year = int(partie)

    # Filtre 3 : post-2020 uniquement
    if not year or year < 2020:
        continue

    rapports.append({
        "title": title,
        "url": BASE_URL + href,
        "year": year,
        "institution": "IGF",
        "fichier": unquote(href.split("/")[-1])
    })

print(f"{len(rapports)} rapports trouvés")
for r in rapports:
    print(r)

9 rapports trouvés
{'title': "Face à la gravité de la situation financière des hôpitaux publics, renforcer l'efficience par une intégration territoriale", 'url': 'https://www.igf.finances.gouv.fr/files/live/sites/igf/files/contributed/Rapports%20de%20mission/2026/2025-E-023%20Rapport.pdf', 'year': 2026, 'institution': 'IGF', 'fichier': '2025-E-023 Rapport.pdf'}
{'title': 'Modèle économique des établissements publics de l’enseignement supérieur', 'url': 'https://www.igf.finances.gouv.fr/files/live/sites/igf/files/contributed/Rapports%20de%20mission/2026/2024-M-050-03%20Rapport%20%20Mod%c3%a8le%20EPES%20WEB.pdf', 'year': 2026, 'institution': 'IGF', 'fichier': '2024-M-050-03 Rapport  Modèle EPES WEB.pdf'}
{'title': 'Évaluation finale du programme NANO 2022', 'url': 'https://www.igf.finances.gouv.fr/files/live/sites/igf/files/contributed/Rapports%20de%20mission/2024/Rapport%20WEB%20-%20biff%c3%a9.pdf', 'year': 2024, 'institution': 'IGF', 'fichier': 'Rapport WEB - biffé.pdf'}
{'title': 'Éva

In [5]:
# Combien de liens au total sur la page ?
print(f"Total liens PDF sur la page : {len(liens_pdf)}")

# Regardons s'il y a une pagination
pagination = soup.find_all("a", href=lambda h: h and "page" in str(h).lower())
for p in pagination[:10]:
    print(p.get("href"), "|", p.text.strip())

Total liens PDF sur la page : 25
/sites/igf/accueil/nos-activites-1/modele-page-pourquoi-igf-2.html | Pourquoi faire appel à l'IGF ?
/liste-de-tous-les-rapports-de-mi.html?beginc472a18f-ec7d-4ef7-a824-53e95a04e575=10&endc472a18f-ec7d-4ef7-a824-53e95a04e575=19&pagesizec472a18f-ec7d-4ef7-a824-53e95a04e575=10& | 2
/liste-de-tous-les-rapports-de-mi.html?beginc472a18f-ec7d-4ef7-a824-53e95a04e575=20&endc472a18f-ec7d-4ef7-a824-53e95a04e575=29&pagesizec472a18f-ec7d-4ef7-a824-53e95a04e575=10& | 3
/liste-de-tous-les-rapports-de-mi.html?beginc472a18f-ec7d-4ef7-a824-53e95a04e575=30&endc472a18f-ec7d-4ef7-a824-53e95a04e575=39&pagesizec472a18f-ec7d-4ef7-a824-53e95a04e575=10& | 4
/liste-de-tous-les-rapports-de-mi.html?beginc472a18f-ec7d-4ef7-a824-53e95a04e575=40&endc472a18f-ec7d-4ef7-a824-53e95a04e575=49&pagesizec472a18f-ec7d-4ef7-a824-53e95a04e575=10& | 5
/liste-de-tous-les-rapports-de-mi.html?beginc472a18f-ec7d-4ef7-a824-53e95a04e575=50&endc472a18f-ec7d-4ef7-a824-53e95a04e575=59&pagesizec472a18f-ec7

In [6]:
# Y a-t-il des liens vers d'autres pages de rapports ?
autres_pages = soup.find_all("a", href=lambda h: h and "rapport" in str(h).lower())
for p in autres_pages[:10]:
    print(p.get("href"), "|", p.text.strip())

/liste-de-tous-les-rapports-de-mi.html | Rapports de mission
/sites/igf/accueil/nos-activites-1/nos-rapports-dactivite.html | Nos rapports d'activité
/liste-de-tous-les-rapports-de-mi.html?beginc472a18f-ec7d-4ef7-a824-53e95a04e575=10&endc472a18f-ec7d-4ef7-a824-53e95a04e575=19&pagesizec472a18f-ec7d-4ef7-a824-53e95a04e575=10& | 2
/liste-de-tous-les-rapports-de-mi.html?beginc472a18f-ec7d-4ef7-a824-53e95a04e575=20&endc472a18f-ec7d-4ef7-a824-53e95a04e575=29&pagesizec472a18f-ec7d-4ef7-a824-53e95a04e575=10& | 3
/liste-de-tous-les-rapports-de-mi.html?beginc472a18f-ec7d-4ef7-a824-53e95a04e575=30&endc472a18f-ec7d-4ef7-a824-53e95a04e575=39&pagesizec472a18f-ec7d-4ef7-a824-53e95a04e575=10& | 4
/liste-de-tous-les-rapports-de-mi.html?beginc472a18f-ec7d-4ef7-a824-53e95a04e575=40&endc472a18f-ec7d-4ef7-a824-53e95a04e575=49&pagesizec472a18f-ec7d-4ef7-a824-53e95a04e575=10& | 5
/liste-de-tous-les-rapports-de-mi.html?beginc472a18f-ec7d-4ef7-a824-53e95a04e575=50&endc472a18f-ec7d-4ef7-a824-53e95a04e575=59&pag

In [7]:
import time

BASE_URL = "https://www.igf.finances.gouv.fr"
PAGE_ID = "c472a18f-ec7d-4ef7-a824-53e95a04e575"

def scrape_page_igf(begin, end):
    url = f"{BASE_URL}/liste-de-tous-les-rapports-de-mi.html?begin{PAGE_ID}={begin}&end{PAGE_ID}={end}&pagesize{PAGE_ID}=10&"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup.find_all("a", href=lambda h: h and ".pdf" in h.lower())

tous_les_rapports = []

for page in range(0, 100, 10):  # 0, 10, 20, ... 90
    print(f"Scraping page begin={page}...")
    liens = scrape_page_igf(page, page+9)

    for lien in liens:
        href = lien.get("href", "")
        title = lien.get("title", "")
        nom_fichier = unquote(href.split("/")[-1]).lower()

        if "annexe" in nom_fichier or "synth" in nom_fichier:
            continue

        parties = href.split("/")
        year = next((int(p) for p in parties if p.isdigit() and len(p) == 4), None)

        if not year or year < 2020:
            continue

        tous_les_rapports.append({
            "title": title,
            "url": BASE_URL + href,
            "year": year,
            "institution": "IGF",
            "fichier": unquote(href.split("/")[-1])
        })

    time.sleep(1)  # on est poli avec le serveur

print(f"\nTotal : {len(tous_les_rapports)} rapports trouvés")

Scraping page begin=0...
Scraping page begin=10...
Scraping page begin=20...
Scraping page begin=30...
Scraping page begin=40...
Scraping page begin=50...
Scraping page begin=60...
Scraping page begin=70...
Scraping page begin=80...
Scraping page begin=90...

Total : 112 rapports trouvés


In [8]:
import pandas as pd

df = pd.DataFrame(tous_les_rapports)

# Vue d'ensemble
print(df["year"].value_counts().sort_index())
print(f"\nDoublons : {df.duplicated(subset='url').sum()}")
print(f"\nTitres vides : {df['title'].eq('').sum()}")

# Aperçu
df.head(10)

year
2021     6
2022    24
2023    22
2024    56
2026     4
Name: count, dtype: int64

Doublons : 56

Titres vides : 0


,title,url,year,institution,fichier
0,Face à la gravité de la situation financière d...,https://www.igf.finances.gouv.fr/files/live/si...,2026,IGF,2025-E-023 Rapport.pdf
1,Modèle économique des établissements publics d...,https://www.igf.finances.gouv.fr/files/live/si...,2026,IGF,2024-M-050-03 Rapport Modèle EPES WEB.pdf
2,Évaluation finale du programme NANO 2022,https://www.igf.finances.gouv.fr/files/live/si...,2024,IGF,Rapport WEB - biffé.pdf
3,Évaluation des contrats et marchés de partenariat,https://www.igf.finances.gouv.fr/files/live/si...,2024,IGF,Version web biffée.pdf
4,Évaluation de l’initiative Tibi,https://www.igf.finances.gouv.fr/files/live/si...,2024,IGF,2025-E-049-03 Rapport TIBI WEB.pdf
5,La maîtrise des frais de justice,https://www.igf.finances.gouv.fr/files/live/si...,2024,IGF,2024-M-073_version WEB_11 juillet 2025_biffé.pdf
6,Rapport sur la formation continue des cadres s...,https://www.igf.finances.gouv.fr/files/live/si...,2024,IGF,2024-M-044-02 Rapport Formation INSP_Version W...
7,Contribution et régulation de la publicité pou...,https://www.igf.finances.gouv.fr/files/live/si...,2024,IGF,2024-M-031-03 Rapport comm-1.pdf
8,Le financement des autorités organisatrices de...,https://www.igf.finances.gouv.fr/files/live/si...,2024,IGF,2024-M-040-03 Rapport Financement AOM_WEB.pdf
9,Face à la gravité de la situation financière d...,https://www.igf.finances.gouv.fr/files/live/si...,2026,IGF,2025-E-023 Rapport.pdf


In [9]:
df = df.drop_duplicates(subset="url")
print(f"Après déduplication : {len(df)} rapports")

Après déduplication : 56 rapports


In [10]:
print(df["year"].value_counts().sort_index())
print(f"\nAperçu des titres :")
print(df["title"].head(10).tolist())

year
2021     3
2022    12
2023    11
2024    28
2026     2
Name: count, dtype: int64

Aperçu des titres :
["Face à la gravité de la situation financière des hôpitaux publics, renforcer l'efficience par une intégration territoriale", 'Modèle économique des établissements publics de l’enseignement supérieur', 'Évaluation finale du programme NANO 2022', 'Évaluation des contrats et marchés de partenariat', 'Évaluation de l’initiative Tibi', 'La maîtrise des frais de justice', 'Rapport sur la formation continue des cadres supérieurs de l’État et le rôle de l’INSP', 'Contribution et régulation de la publicité pour une consommation plus durable', 'Le financement des autorités organisatrices de la mobilité', '2024-M-033-02 Rapport Cotisations sociales OM_WEB.pdf']


In [14]:
from langchain_community.document_loaders import PyPDFLoader
from rag_public_reports.config import DATA_DIR

# Prends un PDF que tu as déjà dans data/
pdf_path = DATA_DIR / "raw" / "2024_Bilan-Instituts-Carnot_IGF.pdf"

loader = PyPDFLoader(pdf_path)
pages = loader.load()

# On envoie seulement les 3 premières pages à Claude
extrait = "\n\n".join([p.page_content for p in pages[:3]])
print(extrait[:2000])  # aperçu

Évaluation du dispositif des instituts Carnot
DÉCEMBRE 2024 
Maxence LANGLOIS-BERTHELOT 
Alexandra BESLY 
Rémy SLOVE 
Nicolas FIORUCCI
Joé VINCENT-GALTIÉ
Émilie-Pauline GALLIÉ 
Guillaume TRONCHET
Christophe RAVIER  
Francis JUTAND

RAPPORT 
ÉVALUATION DU DISPOSITIF DES INSTITUTS CARNOT 
 Étab
li par 
- DÉCEMBRE 2024 -
INSPECTION GÉNÉRALE DES 
FINANCES
N° 2024-M-047-03 
INSPECTION GÉNÉRALE DE 
L’EDUCATION, DU SPORT ET DE LA 
RECHERCHE 
N° 23-24-278 
CONSEIL GÉNÉRAL DE 
L’ÉCONOMIE, DE L’INDUSTRIE, DE 
L’ÉNERGIE ET DES TECHNOLOGIES 
N° 2024/10/CGE/SG 
ÉMILIE-PAULINE GALLIÉ 
Inspectrice générale de 
l’éducation, du sport et de 
la recherche 
GUILLAUME TRONCHET 
Inspecteur général de 
l’éducation, du sport et de 
la recherche 
CHRISTOPHE RAVIER 
Ingénieur général des mines 
FRANCIS JUTAND 
Membre associé du Conseil 
ALEXANDRA BESLY 
Inspectrice des finances 
RÉMY SLOVE 
Inspecteur des finances 
Avec la participation de
NICOLAS FIORUCCI 
Inspecteur stagiaire des finances 
Avec le concours de

In [21]:
import anthropic
import json
import os

client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

from rag_public_reports.config import KNOWN_INSTITUTIONS, KNOWN_THEMES

prompt = f"""Tu es un assistant qui extrait des métadonnées de rapports publics français.

Voici le début d'un rapport :

{extrait[:3000]}

Extrais les informations suivantes et réponds UNIQUEMENT en JSON valide, sans aucun texte autour.

Contraintes STRICTES :
- "institution" : choisis UNE SEULE valeur parmi : {KNOWN_INSTITUTIONS}
- "theme" : choisis UNE SEULE valeur parmi : {KNOWN_THEMES}
- Si plusieurs institutions sont impliquées, prends la principale (ex: IGF si rapport IGF+IGAS)
- "year" : année de publication (entier)
- "title" : titre complet du rapport

Format attendu :
{{
    "title": "...",
    "institution": "...",
    "year": 2024,
    "theme": "..."
}}
"""

response = client.messages.create(
    model="claude-sonnet-4-6",
    max_tokens=500,
    messages=[{"role": "user", "content": prompt}]
)

# Parser le JSON retourné
metadata = json.loads(response.content[0].text)
print(metadata)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [22]:
# Regardons la réponse brute avant de parser
print(repr(response.content[0].text))

'```json\n{\n    "title": "Évaluation du dispositif des instituts Carnot",\n    "institution": "IGF",\n    "year": 2024,\n    "theme": "finances publiques"\n}\n```'


In [23]:
raw = response.content[0].text.strip()
# Nettoie les ```json ... ```
if "```" in raw:
    raw = raw.split("```")[1]
    if raw.startswith("json"):
        raw = raw[4:]
raw = raw.strip()

metadata = json.loads(raw)
print(metadata)

{'title': 'Évaluation du dispositif des instituts Carnot', 'institution': 'IGF', 'year': 2024, 'theme': 'finances publiques'}


In [26]:
import sys
sys.path.insert(0, "../src")  # pour trouver rag_public_reports

from rag_public_reports.catalogue import extraire_metadata, ajouter_au_catalogue
pdf_path = DATA_DIR / "raw" / "2024_Bilan-Instituts-Carnot_IGF.pdf"

# Test sur un PDF
metadata = extraire_metadata(pdf_path)
print(metadata)

ajouter_au_catalogue(metadata, catalogue_path="../data/raw/catalogue.csv")

{'title': 'Évaluation du dispositif des instituts Carnot', 'institution': 'IGF', 'year': 2024, 'theme': 'éducation', 'fichier': '2024_Bilan-Instituts-Carnot_IGF.pdf'}
✅ Ajouté : Évaluation du dispositif des instituts Carnot
